# Archive Search with OpenCLIP and LanceDB

In this example we'll build a Arxiv Search or a recommender based on semantic search using LanceDB. We'll also compare the results with keyword based saerch on Nomic's atlast


## OpenCLIP

![CLIP (1)](https://github.com/lancedb/vectordb-recipes/assets/15766192/11b3b900-0bcb-4a4a-8fd4-804611c85972)


OpenCLIP an open source implementation of OpenAI's CLIP (Contrastive Language-Image Pre-training) as is available with various backends

In [1]:
# SETUP
!pip install lancedb open_clip_torch arxiv --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 6.2.0 requires pyarrow<13,>=2, but you have pyarr

In [2]:
!pip install pandas

## Creating table from arxiv API

### Embedding Paper Summary using CLIP


In [3]:
import torch
import open_clip
import pandas as pd
from open_clip import tokenizer
from tqdm import tqdm
from collections import defaultdict
import arxiv
import lancedb

def embed_func_clip(text):
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
    tokenizer = open_clip.get_tokenizer('ViT-B-32')
    with torch.no_grad():
        text_features = model.encode_text(tokenizer(text))
    return text_features

### Create a DataFrame of the desired length

Here we'll use arxiv python utility to interact with arxiv api and get the document data

In [4]:
def get_arxiv_df(embed_func, length=100):
    results = arxiv.Search(
      query= "cat:cs.AI OR cat:cs.CV OR cat:stat.ML",
      max_results = length,
      sort_by = arxiv.SortCriterion.Relevance,
      sort_order = arxiv.SortOrder.Descending
    ).results()
    df = defaultdict(list)
    for result in tqdm(results, total=length):
        try:
            df["title"].append(result.title)
            df["summary"].append(result.summary)
            df["authors"].append(str(result.authors))
            df["url"].append(result.entry_id)
            df["vector"].append(embed_func(result.summary).tolist()[0])

        except Exception as e:
            print("error: ", e)

    return pd.DataFrame(df)

In [5]:
LENGTH = 100 # Reduce the size for demo
def create_table():
    db = lancedb.connect("db")
    df = get_arxiv_df(embed_func_clip, LENGTH)

    tbl = db.create_table("arxiv", data=df, mode="overwrite")

    return tbl

In [6]:
tbl = create_table()

<ipython-input-4-32209edb23b5>:7: DeprecationWarning: The '(Search).results' method is deprecated, use 'Client.results' instead
  ).results()
  0%|          | 0/100 [00:00<?, ?it/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

100%|██████████| 100/100 [05:30<00:00,  3.30s/it]


In [7]:
import lancedb

db = lancedb.connect("db")

if "arxiv" not in db.table_names():
    tbl = create_table()
else:
    tbl = db.open_table("arxiv")


In [8]:
len(tbl)

100

## Semantic Search by concepts or summary

In [9]:
from IPython.display import display, HTML

def search_table(query, embed_func=embed_func_clip, lim=3):
    db = lancedb.connect("db")
    tbl = db.open_table("arxiv")

    embs = embed_func(query)

    return tbl.search(embs.tolist()[0]).limit(3).to_df()


In [10]:
len(tbl)

100

In [11]:
# MobileSAM paper abstract 2nd half
query = """
Many of such applications need to be run on resource-constraint edge devices,
like mobile phones. In this work, we aim to make SAM mobile-friendly by replacing the heavyweight
image encoder with a lightweight one. A naive way to train such a new SAM as in the original SAM
paper leads to unsatisfactory performance, especially when limited training sources are available. We
find that this is mainly caused by the coupled optimization of the image encoder and mask decoder,
motivated by which we propose decoupled distillation. Concretely, we distill the knowledge from
the heavy image encoder (ViT-H in the original SAM) to a lightweight image encoder, which can be
automatically compatible with the mask decoder in the original SAM. The training can be completed
on a single GPU within less than one day, and the resulting lightweight SAM is termed MobileSAM
which is more than 60 times smaller yet performs on par with the original SAM. For inference speed,
With a single GPU, MobileSAM runs around 10ms per image: 8ms on the image encoder and 4ms
on the mask decoder. With superior performance, our MobileSAM is around 5 times faster than the
concurrent FastSAM and 7 times smaller, making it more suitable for mobile applications. Moreover,
we show that MobileSAM can run relatively smoothly on CPU
"""

result = search_table(query)

result.pop("vector")
display(HTML(result.to_html()))


<ipython-input-9-d07584fcae98>:9: DeprecatedWarning: to_df is deprecated as of 0.3.1 and will be removed in 0.4.0. Use to_pandas() instead
  return tbl.search(embs.tolist()[0]).limit(3).to_df()


,title,summary,authors,url,_distance
0,There Are Many Consistent Explanations of Unlabeled Data: Why You Should Average,"Presently the most successful approaches to semi-supervised learning are\nbased on consistency regularization, whereby a model is trained to be robust to\nsmall perturbations of its inputs and parameters. To understand consistency\nregularization, we conceptually explore how loss geometry interacts with\ntraining procedures. The consistency loss dramatically improves generalization\nperformance over supervised-only training; however, we show that SGD struggles\nto converge on the consistency loss and continues to make large steps that lead\nto changes in predictions on the test data. Motivated by these observations, we\npropose to train consistency-based methods with Stochastic Weight Averaging\n(SWA), a recent approach which averages weights along the trajectory of SGD\nwith a modified learning rate schedule. We also propose fast-SWA, which further\naccelerates convergence by averaging multiple points within each cycle of a\ncyclical learning rate schedule. With weight averaging, we achieve the best\nknown semi-supervised results on CIFAR-10 and CIFAR-100, over many different\nquantities of labeled training data. For example, we achieve 5.0% error on\nCIFAR-10 with only 4000 labels, compared to the previous best result in the\nliterature of 6.3%.","[arxiv.Result.Author('Ben Athiwaratkun'), arxiv.Result.Author('Marc Finzi'), arxiv.Result.Author('Pavel Izmailov'), arxiv.Result.Author('Andrew Gordon Wilson')]",http://arxiv.org/abs/1806.05594v3,39.569557
1,XFlow: Cross-modal Deep Neural Networks for Audiovisual Classification,"In recent years, there have been numerous developments towards solving\nmultimodal tasks, aiming to learn a stronger representation than through a\nsingle modality. Certain aspects of the data can be particularly useful in this\ncase - for example, correlations in the space or time domain across modalities\n- but should be wisely exploited in order to benefit from their full predictive\npotential. We propose two deep learning architectures with multimodal\ncross-connections that allow for dataflow between several feature extractors\n(XFlow). Our models derive more interpretable features and achieve better\nperformances than models which do not exchange representations, usefully\nexploiting correlations between audio and visual data, which have a different\ndimensionality and are nontrivially exchangeable. Our work improves on existing\nmultimodal deep learning algorithms in two essential ways: (1) it presents a\nnovel method for performing cross-modality (before features are learned from\nindividual modalities) and (2) extends the previously proposed\ncross-connections which only transfer information between streams that process\ncompatible data. Illustrating some of the representations learned by the\nconnections, we analyse their contribution to the increase in discrimination\nability and reveal their compatibility with a lip-reading network intermediate\nrepresentation. We provide the research community with Digits, a new dataset\nconsisting of three data types extracted from videos of people saying the\ndigits 0-9. Results show that both cross-modal architectures outperform their\nbaselines (by up to 11.5%) when evaluated on the AVletters, CUAVE and Digits\ndatasets, achieving state-of-the-art results.","[arxiv.Result.Author('Cătălina Cangea'), arxiv.Result.Author('Petar Veličković'), arxiv.Result.Author('Pietro Liò')]",http://arxiv.org/abs/1709.00572v2,40.346905
2,A Hybrid Instance-based Transfer Learning Method,"In recent years, supervised machine learning models have demonstrated\ntremendous success in a variety of application domains. Despite the promising\nresults, these successful models are data hungry and their performance relies\nheavily on the size of training data. However, in many healthcare applications\nit is difficult to collect sufficiently large training datasets. Tra

In [12]:
# Exmaple 2: Search via a concept you're reading
query = """
What is the general idea behind self-supervised learning.
"""

result = search_table(query)

result.pop("vector")
display(HTML(result.to_html()))


<ipython-input-9-d07584fcae98>:9: DeprecatedWarning: to_df is deprecated as of 0.3.1 and will be removed in 0.4.0. Use to_pandas() instead
  return tbl.search(embs.tolist()[0]).limit(3).to_df()


,title,summary,authors,url,_distance
0,Is 'Unsupervised Learning' a Misconceived Term?,"Is all of machine learning supervised to some degree? The field of machine\nlearning has traditionally been categorized pedagogically into\n$supervised~vs~unsupervised~learning$; where supervised learning has typically\nreferred to learning from labeled data, while unsupervised learning has\ntypically referred to learning from unlabeled data. In this paper, we assert\nthat all machine learning is in fact supervised to some degree, and that the\nscope of supervision is necessarily commensurate to the scope of learning\npotential. In particular, we argue that clustering algorithms such as k-means,\nand dimensionality reduction algorithms such as principal component analysis,\nvariational autoencoders, and deep belief networks are each internally\nsupervised by the data itself to learn their respective representations of its\nfeatures. Furthermore, these algorithms are not capable of external inference\nuntil their respective outputs (clusters, principal components, or\nrepresentation codes) have been identified and externally labeled in effect. As\nsuch, they do not suffice as examples of unsupervised learning. We propose that\nthe categorization `supervised vs unsupervised learning' be dispensed with, and\ninstead, learning algorithms be categorized as either\n$internally~or~externally~supervised$ (or both). We believe this change in\nperspective will yield new fundamental insights into the structure and\ncharacter of data and of learning algorithms.",[arxiv.Result.Author('Stephen G. Odaibo')],http://arxiv.org/abs/1904.03259v1,30.549643
1,Quantizing Convolutional Neural Networks for Low-Power High-Throughput Inference Engines,"Deep learning as a means to inferencing has proliferated thanks to its\nversatility and ability to approach or exceed human-level accuracy. These\ncomputational models have seemingly insatiable appetites for computational\nresources not only while training, but also when deployed at scales ranging\nfrom data centers all the way down to embedded devices. As such, increasing\nconsideration is being made to maximize the computational efficiency given\nlimited hardware and energy resources and, as a result, inferencing with\nreduced precision has emerged as a viable alternative to the IEEE 754 Standard\nfor Floating-Point Arithmetic. We propose a quantization scheme that allows\ninferencing to be carried out using arithmetic that is fundamentally more\nefficient when compared to even half-precision floating-point. Our quantization\nprocedure is significant in that we determine our quantization scheme\nparameters by calibrating against its reference floating-point model using a\nsingle inference batch rather than (re)training and achieve end-to-end post\nquantization accuracies comparable to the reference model.","[arxiv.Result.Author('Sean O. Settle'), arxiv.Result.Author('Manasa Bollavaram'), arxiv.Result.Author(""Paolo D'Alberto""), arxiv.Result.Author('Elliott Delaye'), arxiv.Result.Author('Oscar Fernandez'), arxiv.Result.Author('Nicholas Fraser'), arxiv.Result.Author('Aaron Ng'), arxiv.Result.Author('Ashish Sirasao'), arxiv.Result.Author('Michael Wu')]",http://arxiv.org/abs/1805.07941v1,33.038021
2,Task-Free Continual Learning,"Methods proposed in the literature towards continual deep learning typically\noperate in a task-based sequential learning setup. A sequence of tasks is\nlearned, one at a time, with all data of current task available but not of\nprevious or future tasks. Task boundaries and identities are known at all\ntimes. This setup, however, is rarely encountered in practical applications.\nTherefore we investigate how to transform continual learning to an online\nsetup. We develop a system that keeps on learning over time in a streaming\nfashion, with data distributions gradually changing and without the notion of\nseparate tasks. To this end, we build on the work on Memory Aware Synapses, and\nshow how th